# Decision Tree Classifier

In [1]:
import math
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
class Decision_Tree_Classifier:
    """
    Class for building decision tree classifier
    """
    def __init__(self, y, X, min_samples_split=20, max_depth=5, depth=0, node_type='root', rule="", criteria=""):
        self.y, self.X = y, X
        self.features = [col for col in self.X.columns]
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth 
        self.depth = depth if depth else 0
        self.node_type = node_type
        self.rule = rule 
        self.criteria=criteria
        self.gini_or_entropy = self.get_gini_or_entropy(y1_count=None, y2_count=None, criteria=self.criteria)

        # predicting y for a node
        self.y_counts = Counter(self.y)
        counts_sorted = list(sorted(self.y_counts.items(), key=lambda item: item[1]))
        self.y_pred = None
        if len(counts_sorted) > 0:
            self.y_pred = counts_sorted[-1][0]

        self.size_of_node = len(y)
        self.left_node, self.right_node  = None, None
        self.best_feature, self.best_threshold = None, None

    @staticmethod
    def moving_average(X, window):
        """
        Calculates the moving average, that will be used as list of thresholds
        """
        return np.convolve(X, np.ones(window), 'valid') / window

    def get_gini_or_entropy(self, y1_count=None, y2_count=None, criteria='gini'):
        """
        Returns gini or entropy of a node 
        """
        if not y1_count and not y2_count:
            self.y_counts = Counter(self.y)
            y1_count, y2_count = self.y_counts.get(0, 0), self.y_counts.get(1, 0)
        if (y1_count + y2_count) == 0: # to avoid 'division by 0'
            return 0.0
        p1 = y1_count / (y1_count + y2_count)
        p2 = y2_count / (y1_count + y2_count)

        if self.criteria == 'gini':
            gini_impurity = 1 - (p1 ** 2 + p2 ** 2)
            return gini_impurity
        else:
            if p1 != 0 and p2 != 0:
                entropy = round(-1 *(p1 * math.log(p1, 2) + p2 * math.log(p2,2)), 3)
            elif p1 == 0 and p2 != 0:
                entropy = round(-1 *(p2 * math.log(p2,2)), 3)
            elif p1 != 0 and p2 ==0:
                entropy = round(-1 *(p1 * math.log(p1, 2)), 3)
            else:
                entropy = 0.0
            return abs(entropy)

    def determine_best_split(self):
        """
        determines the best feature and threshold for splitting a node
        """
        df = self.X.copy()
        df['y'] = self.y
        base_gini_or_entropy = self.get_gini_or_entropy(y1_count=None, y2_count=None, criteria=self.criteria)

        # Finding the split that gives the best information gain
        max_gain, best_feature, best_threshold = 0, None, None

        for feature in self.features:
            Xdf = df.dropna().sort_values(feature)
            thresholds = self.moving_average(Xdf[feature].unique(), 2)

            for threshold in thresholds:
                # Spliting the dataset 
                left_node_counts = Counter(Xdf[Xdf[feature]<threshold]['y']) # returns a Counter object, not same as left_node_size
                right_node_counts = Counter(Xdf[Xdf[feature]>=threshold]['y'])
                y0_left, y1_left, y0_right, y1_right = left_node_counts.get(0, 0), left_node_counts.get(1, 0), right_node_counts.get(0, 0), right_node_counts.get(1, 0)

                # Getting gini impurities and weight of left node and right node
                left_node_gini_or_entropy = self.get_gini_or_entropy(y1_count=y0_left, y2_count=y1_left, criteria=self.criteria)
                right_node_gini_or_entropy = self.get_gini_or_entropy(y1_count=y0_right, y2_count=y1_right, criteria=self.criteria)

                left_node_size, right_node_size = y0_left + y1_left, y0_right + y1_right
                left_node_weight, right_node_weight = left_node_size / (left_node_size + right_node_size), right_node_size / (left_node_size + right_node_size)

                # Calculating the weighted GINI impurity or entropy
                weighted_gini_or_entropy = left_node_weight * left_node_gini_or_entropy + right_node_weight * right_node_gini_or_entropy
                information_gain = base_gini_or_entropy - weighted_gini_or_entropy

                # Checking if this is the best split so far 
                if information_gain > max_gain:
                    best_feature = feature
                    best_threshold = threshold 
                    max_gain = information_gain # Updating maximum gain

        return (best_feature, best_threshold)


      def build_decision_tree(self):
        """
        Recursive method to grow the decision tree
        """
        # Making a df from the data 
        df = self.X.copy()
        df['y'] = self.y

        if (self.depth < self.max_depth) and (self.size_of_node >= self.min_samples_split):

            self.best_feature, self.best_threshold = self.determine_best_split()

            if self.best_feature:
                # Getting the left and right nodes
                left_df, right_df = df[df[self.best_feature]<=self.best_threshold].copy(), df[df[self.best_feature]>self.best_threshold].copy()

                # Creating the left and right nodes
                left_node = Decision_Tree_Classifier(
                    left_df['y'].values.tolist(), 
                    left_df[self.features], 
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth, 
                    depth=self.depth + 1, 
                    node_type='left_node',
                    rule=f"{self.best_feature} <= {round(self.best_threshold, 3)}",
                    criteria=self.criteria
                    )

                self.left_node = left_node 
                self.left_node.build_decision_tree()

                right_node = Decision_Tree_Classifier(
                    right_df['y'].values.tolist(), 
                    right_df[self.features], 
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth, 
                    depth=self.depth + 1,                     
                    node_type='right_node',
                    rule=f"{self.best_feature} > {round(self.best_threshold, 3)}",
                    criteria=self.criteria
                    )

                self.right_node = right_node
                self.right_node.build_decision_tree()

    def predict(self, X):
        """
        Batch prediction method 
        """
        predictions = list()
        for val, x in X.iterrows():
            values = dict()
            for feature in self.features:
                values.update({feature: x[feature]})

            cur_node = self        
            while cur_node.depth < cur_node.max_depth:
                best_feature, best_threshold = cur_node.best_feature, cur_node.best_threshold
                if cur_node.size_of_node < cur_node.min_samples_split:
                    break 
                if not cur_node.best_feature:
                    break
                if (values[cur_node.best_feature] < cur_node.best_threshold):
                    if self.left_node:
                        cur_node = cur_node.left_node
                else:
                    if self.right_node:
                        cur_node = cur_node.right_node
            predictions.append(cur_node.y_pred)
        return predictions

    def print_tree(self):
        """
        print decision tree
        """
        padding = int(self.depth * 8)
        space_bar = "-" * padding
        if self.node_type == 'root':
            print("Root Node")
        else:
            print(f"|{space_bar} Splitting Rule: {self.rule}")

        if self.criteria == 'gini':
            print(f"{' ' * padding}   | Gini Impurity : {round(self.gini_or_entropy, 3)}")
        else:
            print(f"{' ' * padding}   | Entropy : {round(self.gini_or_entropy, 3)}")
        print(f"{' ' * padding}   | Class distribution : {dict(self.y_counts)}")
        print(f"{' ' * padding}   | Predicted value : {self.y_pred}")

        if self.left_node: 
            self.left_node.print_tree()

        if self.right_node:
            self.right_node.print_tree()

# Decision Tree for Odd and Even

In [13]:
# Reading data for classification 
data_2 =  pd.read_excel("/content/odd_even.xlsx")
data_2

,x3,x2,x1,x0,y
0,0,0,0,0,0
1,0,0,0,1,1
2,0,0,1,0,0
3,0,0,1,1,1
4,0,1,0,0,0
5,0,1,0,1,1
6,0,1,1,0,0
7,0,1,1,1,1
8,1,0,0,0,0
9,1,0,0,1,1


In [14]:
X = data_2.drop("y", axis=1)
y = data_2["y"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=47)

In [15]:
dtree_entropy = Decision_Tree_Classifier(y_train, X_train, min_samples_split=2, criteria='entropy')

dtree_entropy.build_decision_tree()
dtree_entropy.print_tree()

Root Node
   | Entropy : 1.0
   | Class distribution : {0: 6, 1: 6}
   | Predicted value : 1
|-------- Splitting Rule: x0 <= 0.5
           | Entropy : 0.0
           | Class distribution : {0: 6}
           | Predicted value : 0
|-------- Splitting Rule: x0 > 0.5
           | Entropy : 0.0
           | Class distribution : {1: 6}
           | Predicted value : 1


In [17]:
# Making predictions
y_pred_test = dtree_entropy.predict(X_test)
y_pred_train = dtree_entropy.predict(X_train)

# Measurring accuracy
print(f"The train accuracy: {round(accuracy_score(y_train, y_pred_train),3)}")
print(f"The test accuracy: {round(accuracy_score(y_test, y_pred_test),3)}")

The train accuracy: 1.0
The test accuracy: 1.0


# Spam Dataset

In [2]:
# Reading data for classification 
data =  pd.read_excel("/content/spam_data.xls", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.64,0.00,0.00,0.00,0.32,0.00,1.29,1.93,0.00,0.96,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.00,1.59,0.0,0.43,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.00,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.12,0.0,0.06,0.06,0.0,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [3]:
column_names = dict()
for i,col in enumerate(data.columns):
  column_names[i] = "feature_" + str(i+1)
data.rename(columns=column_names, inplace=True)

X = data.drop("feature_58", axis=1)
y = data["feature_58"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=47)

print(y_train.sum()/len(y_train))

# Decision Tree with Gini Impurity as splitting Criteria

In [6]:
dtree_gini = Decision_Tree_Classifier(y_train, X_train, min_samples_split=2, max_depth=5, criteria='gini')

dtree_gini.build_decision_tree()
dtree_gini.print_tree()

Root Node
   | Gini Impurity : 0.478
   | Class distribution : {1: 1450, 0: 2230}
   | Predicted value : 0
|-------- Splitting Rule: feature_52 <= 0.08
           | Gini Impurity : 0.267
           | Class distribution : {1: 338, 0: 1796}
           | Predicted value : 0
|---------------- Splitting Rule: feature_7 <= 0.045
                   | Gini Impurity : 0.191
                   | Class distribution : {0: 1775, 1: 213}
                   | Predicted value : 0
|------------------------ Splitting Rule: feature_24 <= 0.01
                           | Gini Impurity : 0.15
                           | Class distribution : {0: 1742, 1: 155}
                           | Predicted value : 0
|-------------------------------- Splitting Rule: feature_16 <= 0.195
                                   | Gini Impurity : 0.117
                                   | Class distribution : {0: 1662, 1: 111}
                                   | Predicted value : 0
|----------------------------------------

In [7]:
# Making predictions
y_pred_test = dtree_gini.predict(X_test)
y_pred_train = dtree_gini.predict(X_train)

# Measurring accuracy
print(f"The train accuracy: {round(accuracy_score(y_train, y_pred_train),3)}")
print(f"The test accuracy: {round(accuracy_score(y_test, y_pred_test),3)}")


The train accuracy: 0.924
The test accuracy: 0.924


# Decision Tree with Entropy as splitting Criteria

In [8]:
dtree_entropy = Decision_Tree_Classifier(y_train, X_train, min_samples_split=2, criteria='entropy')

dtree_entropy.build_decision_tree()
dtree_entropy.print_tree()

Root Node
   | Entropy : 0.967
   | Class distribution : {1: 1450, 0: 2230}
   | Predicted value : 0
|-------- Splitting Rule: feature_52 <= 0.08
           | Entropy : 0.63
           | Class distribution : {1: 338, 0: 1796}
           | Predicted value : 0
|---------------- Splitting Rule: feature_7 <= 0.045
                   | Entropy : 0.491
                   | Class distribution : {0: 1775, 1: 213}
                   | Predicted value : 0
|------------------------ Splitting Rule: feature_53 <= 0.09
                           | Entropy : 0.403
                           | Class distribution : {0: 1734, 1: 151}
                           | Predicted value : 0
|-------------------------------- Splitting Rule: feature_25 <= 0.135
                                   | Entropy : 0.53
                                   | Class distribution : {0: 1074, 1: 147}
                                   | Predicted value : 0
|---------------------------------------- Splitting Rule: feature_56 <= 

In [9]:
# Making predictions
y_pred_test = dtree_entropy.predict(X_test)
y_pred_train = dtree_entropy.predict(X_train)

# Measurring accuracy
print(f"The train accuracy: {round(accuracy_score(y_train, y_pred_train),3)}")
print(f"The test accuracy: {round(accuracy_score(y_test, y_pred_test),3)}")


The train accuracy: 0.917
The test accuracy: 0.922
